In [1]:
import xml.etree.ElementTree as ET
from PIL import Image
import os
import pandas as pd
import numpy as np
import json
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.data import random_split, DataLoader, Dataset

import torch
from torchvision import datasets, transforms

In [ ]:
%pip install -q transformers

Найдем все json файлы с аннотациями

In [2]:
anno_pathes_json = []
file_name = '.json'

absolute_path = 'C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\data\\BallerTV'
crop_path = 'C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\crops'

if not os.path.exists(crop_path):
    os.makedirs(crop_path)

for root, dirs, files in os.walk(absolute_path):    
    for file in files:
        # проверяем, есть ли заданный файл в списке файлов текущей папки
        if (file_name in file) & ('.jsonl' not in file):
            # если есть, то добавляем путь в список
            anno_pathes_json.append(os.path.join(root, file)) 

Делаем кроп и сохраняем файлы и аннотации с номерами играков

In [3]:
alpha = 0.1
num = 0
annos_list = []
anno_dict = {'file_name': [], 'text': []}
pathes_list = []

for anno in tqdm(anno_pathes_json):
    with open(anno, 'r') as f:
        data = json.load(f)

    #Преобразуем json  в dataFrame
    points = pd.read_json(json.dumps(data['shapes']))
    x = pd.read_json(anno, orient='index')
    frame = x.iloc[3, :][0]

    img_path = anno.replace('anno', 'frames')
    img_path = img_path.replace('.json', '.jpeg')
    img = Image.open(img_path)

    for j in range(len(points)):
        out_file = {}
        point_obj = pd.DataFrame(points.iloc[j, 0:2])
        point = point_obj.iloc[1, :]
        label = point_obj.iloc[0, :]
        split_label = label[j].split('_')
        number = split_label[-1]

        for box in point:   

            if number.isdigit():

                xtl, ytl, xbr, ybr = int(box[0][0]), int(box[0][1]), int(box[1][0]), int(box[1][1])
                            
                x_dif = np.abs(xtl - xbr)
                y_dif = np.abs(ytl - ybr)

                xtl = xtl - alpha * (x_dif / 2)
                xbr = xbr + alpha * (x_dif / 2)
                ytl = ytl - alpha * (y_dif / 2)
                ybr = ybr + alpha * (y_dif / 2)

                # Сделаем кроп номера на футболке
                croppped_image = img.crop((xtl, ytl, xbr, ybr))
                full_path = crop_path + '\\' + str(number)                

                if not os.path.exists(full_path):
                    os.makedirs(full_path)    
                    #print(f'Папка {number} создана')  
                                  
                res_path = full_path +'\\' + f'ballerTV_{str(num)}example.jpg'
                num += 1

                annos_list.append(str(number))
                pathes_list.append(res_path )
                anno_dict['text'] = annos_list
                anno_dict['file_name'] = pathes_list
                
                croppped_image.save(res_path)

df = pd.DataFrame(anno_dict, index=None)
df.to_csv("anno_ballerTV.csv")
df.head()
                

   

100%|██████████| 5598/5598 [01:40<00:00, 55.88it/s] 


,file_name,text
0,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,33
1,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,30
2,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,33
3,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,30
4,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,33


Найдем все папки в которых лежит вайл с аннотацией

In [4]:
crop_path = "C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\crops"
absolute_path = "C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\data\\BallerTV"

if not os.path.exists(crop_path):
    os.makedirs(crop_path)

file_name = 'annotations.xml'

anno_pathses = []
# пройдемся рекурсивно по всем папкам, начиная с absolute_path
for root, dirs, files in os.walk(absolute_path):    
    # проверяем, есть ли заданный файл в списке файлов текущей папки
    if file_name in files:
        # если есть, то добавляем путь в список
        anno_pathses.append(os.path.join(root, file_name))          

In [8]:
nums = []
num = 0
folder_pathes = []
pathes_list = []
annos_list = []
anno_dict = {'file_name': [], 'text': []}

alpha = 0.2

for path in tqdm(anno_pathses):

    # Загрузим XML файл
    tree = ET.parse(path)

    # Получим корневой элемент XML файла
    root = tree.getroot()

    # Проитерируемся по дочерним элементам корня
    for image in root.findall('image'):
        
        image_id = image.get('id')
        file_name = image.get('name')
        file_name = 'data\\' + file_name
  
        # Переберем все baundingbox`s
        for box in image.findall('box'):
            
            image = Image.open(file_name)
           
            label = box.get('label')

            # Проверим является ли метка меткой номера
            if label == "Jersey":
                xtl = int(float(box.get('xtl')))
                ytl = int(float(box.get('ytl')))
                xbr = int(float(box.get('xbr')))
                ybr = int(float(box.get('ybr')))
                number = box.find('attribute[@name="number"]').text
                if number == '-1':
                    continue
               
                x_dif = np.abs(xtl - xbr)
                y_dif = np.abs(ytl - ybr)

                xtl = xtl - alpha * (x_dif / 2)
                xbr = xbr + alpha * (x_dif / 2)
                ytl = ytl - alpha * (y_dif / 2)
                ybr = ybr + alpha * (y_dif / 2)

                # Сделаем кроп номера на футболке
                croppped_image = image.crop((xtl, ytl, xbr, ybr))
                full_path = crop_path + '\\' + str(number)
                num += 1

                if not os.path.exists(full_path):
                    os.makedirs(full_path)    
                    #print(f'Папка {number} создана')  
                                  
                res_path = full_path +'\\' + f'ballerTV_{str(num)}example.jpg'

                annos_list.append(int(number))
                pathes_list.append(res_path )
                anno_dict['text'] = annos_list
                anno_dict['file_name'] = pathes_list
                
                croppped_image.save(res_path)

df = pd.DataFrame(anno_dict, index=None)
df.to_csv("anno_ballerTV2.csv")
df.head()


100%|██████████| 105/105 [04:08<00:00,  2.37s/it]


,file_name,text
0,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,1
1,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,8
2,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,11
3,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,3
4,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,3


In [9]:
anno_pathes_json = []
file_name = '.json'

absolute_path = 'data\\basketball\\16-11-2022'
crop_path = 'C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\crops'

if not os.path.exists(crop_path):
    os.makedirs(crop_path)

for root, dirs, files in os.walk(absolute_path):    
    for file in files:
        # проверяем, есть ли заданный файл в списке файлов текущей папки
        if (file_name in file) & ('.jsonl' not in file):
            # если есть, то добавляем путь в список
            anno_pathes_json.append(os.path.join(root, file)) 

In [10]:
num = 0
alpha = 0.2
annos_list = []
anno_dict = {'file_name': [], 'text': []}
pathes_list = []

for i, anno in enumerate(anno_pathes_json):
    with open(anno, 'r') as f:
        data = json.load(f)
       

    frames = pd.read_json(json.dumps(data['frames']))
    print(f'{i + 1}ая папка из {len(anno_pathes_json)}')


    for i in tqdm(range(len(frames))):

        img_name = frames['img_name'][i]
        bboxes = frames['detections'][i]
        #print(bboxes)

        anno_path = anno.replace('anno', 'frames')
        anno_path = anno_path.replace('.json', '\\')
        anno_path_fin = anno_path + str(img_name)

        #image = cv2.cvtColor(cv2.imread(anno_path_fin), cv2.COLOR_BGR2RGB)
        img = Image.open(anno_path_fin)

        # Создание объекта ImageDraw для рисования
        #draw = ImageDraw.Draw(image)

        for detection in bboxes:
        
            ocr_jersey = detection['ocr_jersey']
            if ocr_jersey != '-1':
                jersey_bbox_global = detection['jersey_bbox_global']
                xtl = int(jersey_bbox_global[0])
                ytl = int(jersey_bbox_global[1])
                xbr = int(jersey_bbox_global[2])           
                ybr = int(jersey_bbox_global[3])
                
                x_dif = np.abs(xtl - xbr)
                y_dif = np.abs(ytl - ybr)

                xtl = xtl - alpha * (x_dif / 2)
                xbr = xbr + alpha * (x_dif / 2)
                ytl = ytl - alpha * (y_dif / 2)
                ybr = ybr + alpha * (y_dif / 2)

                #draw.rectangle([(jersey_bbox_global[0], jersey_bbox_global[1]), (jersey_bbox_global[2], jersey_bbox_global[3])], outline='red')
                #cv2.rectangle(image, (xtl, ytl), (xbr, ybr), 2)
                croppped_image = img.crop((xtl, ytl, xbr, ybr))
                full_path = crop_path + '\\' + str(ocr_jersey)
                res_path = full_path + '\\' f'basketball_{num}example.jpg'
                num += 1

                if not os.path.exists(full_path):
                        os.makedirs(full_path)    

                annos_list.append(str(ocr_jersey))
                pathes_list.append(res_path )
                anno_dict['text'] = annos_list
                anno_dict['file_name'] = pathes_list

                croppped_image.save(res_path)

df = pd.DataFrame(anno_dict, index=None)
df.to_csv("anno_basketball.csv")
df.head()

1ая папка из 22


100%|██████████| 500/500 [00:07<00:00, 66.49it/s]


2ая папка из 22


100%|██████████| 500/500 [00:05<00:00, 98.41it/s] 


3ая папка из 22


...

21ая папка из 22


100%|██████████| 500/500 [00:07<00:00, 66.22it/s]


22ая папка из 22


100%|██████████| 500/500 [00:04<00:00, 115.47it/s]


,file_name,text
0,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,1
1,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,3
2,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,1
3,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,3
4,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,1


In [11]:
crop_path = "C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\crops"
absolute_path = "C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\data\\FIBA3x3\\fiba3x3_tracking_135"

if not os.path.exists(crop_path):
    os.makedirs(crop_path)

file_name = 'annotations.xml'

anno_pathses = []
# пройдемся рекурсивно по всем папкам, начиная с absolute_path
for root, dirs, files in os.walk(absolute_path):    
    # проверяем, есть ли заданный файл в списке файлов текущей папки
    if file_name in files:
        # если есть, то добавляем путь в список
        anno_pathses.append(os.path.join(root, file_name))  

In [12]:
nums = []
num = 0
folder_pathes = []
pathes_list = []
annos_list = []
anno_dict = {'file_name': [], 'text': []}

alpha = 0.2

for path in tqdm(anno_pathses):

    # Загрузим XML файл
    tree = ET.parse(path)

    # Получим корневой элемент XML файла
    root = tree.getroot()

    # Проитерируемся по дочерним элементам корня
    for image in root.findall('image'):
        
        image_id = image.get('id')
        file_name = image.get('name')
        #file_name = file_name.replace('/', '\\')
        file_name = os.path.join(absolute_path, file_name)
        if 'fiba3x3/images/331/' in file_name:
            file_name = file_name.replace('fiba3x3/images/331', 'frames')
        file_name = file_name.replace('frames', 'frames_135')
        # Переберем все baundingbox`s
        for box in image.findall('box'):
            
            image = Image.open(file_name)
           
            label = box.get('label')

            # Проверим является ли метка меткой номера
            if label == "Jersey":
                xtl = int(float(box.get('xtl')))
                ytl = int(float(box.get('ytl')))
                xbr = int(float(box.get('xbr')))
                ybr = int(float(box.get('ybr')))
                number = box.find('attribute[@name="number"]').text

                x_dif = np.abs(xtl - xbr)
                y_dif = np.abs(ytl - ybr)

                xtl = xtl - alpha * (x_dif / 2)
                xbr = xbr + alpha * (x_dif / 2)
                ytl = ytl - alpha * (y_dif / 2)
                ybr = ybr + alpha * (y_dif / 2)

                # Сделаем кроп номера на футболке
                croppped_image = image.crop((xtl, ytl, xbr, ybr))
                full_path = crop_path + '\\' + str(number)
                num += 1

                if not os.path.exists(full_path):
                    os.makedirs(full_path)    
                    #print(f'Папка {number} создана')  
                                  
                res_path = full_path +'\\' + f'fiba_{str(num)}example.jpg'

                annos_list.append(int(number))
                pathes_list.append(res_path )
                anno_dict['text'] = annos_list
                anno_dict['file_name'] = pathes_list
                
                croppped_image.save(res_path)

df = pd.DataFrame(anno_dict, index=None)
df.to_csv("anno_fiba.csv")
df.head()

100%|██████████| 135/135 [05:18<00:00,  2.36s/it]


,file_name,text
0,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,4
1,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,7
2,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,4
3,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,4
4,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,4


In [13]:
anno_pathes_json = []
img_pathes = {}
file_name = '.json'
img_name = '.jpg'

absolute_path = 'C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\data\\NCAA\\NCAA_1_jersey\\second_task'
crop_path = 'C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\crops'
img_folder_path = 'C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\data\\NCAA\\NCAA_1_tracking\\playerTrackingFrames'

if not os.path.exists(crop_path):
    os.makedirs(crop_path)

for root, dirs, files in os.walk(absolute_path):      
    for file in files:
        # проверяем, есть ли заданный файл в списке файлов текущей папки
        if (file_name in file) & ('.jsonl' not in file):
            # если есть, то добавляем путь в список
            anno_pathes_json.append(os.path.join(root, file)) 

for root, dirs, files in os.walk(img_folder_path):        
    for file in files:
        # проверяем, есть ли заданный файл в списке файлов текущей папки
        if (img_name in file):
            # если есть, то добавляем путь в список
            sh_file = file.replace('frame_', '')
            img_pathes[sh_file] = os.path.join(root, file)  

In [14]:
alpha = 0.1
num = 0
annos_list = []
anno_dict = {'file_name': [], 'text': []}
pathes_list = []

for anno in tqdm(anno_pathes_json):
    with open(anno, 'r') as f:
        data = json.load(f)

    #Преобразуем json  в dataFrame
    points = pd.read_json(json.dumps(data['shapes']))
    img_name = str(json.dumps(data['imagePath']))
    img_path = img_name.replace('"', '')
    img_path = os.path.split(img_path)
    img_path = img_pathes[img_path[-1]]

    img = Image.open(img_path)
    

    for j in range(len(points)):
        out_file = {}    

        if points["shape_type"][j] == "rectangle":                       
        
            label = points['label']
            split_label = label[j].split('_')
            number = split_label.pop()
            
            point_obj = points['points']
            singl_point = point_obj[j]
            [xtl, ytl], [xbr, ybr] = singl_point

            x_dif = np.abs(xtl - xbr)
            y_dif = np.abs(ytl - ybr)

            xtl = xtl - alpha * (x_dif / 2)
            xbr = xbr + alpha * (x_dif / 2)
            ytl = ytl - alpha * (y_dif / 2)
            ybr = ybr + alpha * (y_dif / 2)

            # Сделаем кроп номера на футболке
            croppped_image = img.crop((xtl, ytl, xbr, ybr))
            full_path = crop_path + '\\' + str(number)                

            if not os.path.exists(full_path):
                os.makedirs(full_path)    
                #print(f'Папка {number} создана')  
                                  
            res_path = full_path +'\\' + f'ncaa_{str(num)}example.jpg'
            num += 1

            annos_list.append(str(number))
            pathes_list.append(res_path )
            anno_dict['text'] = annos_list
            anno_dict['file_name'] = pathes_list
                
            croppped_image.save(res_path)

df = pd.DataFrame(anno_dict, index=None)
df.to_csv("anno_ncaa.csv")
df.head()  

100%|██████████| 6965/6965 [01:26<00:00, 80.31it/s] 


,file_name,text
0,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,11
1,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,0
2,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,33
3,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,12
4,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,9


In [15]:
df1 = pd.read_csv('anno_ballerTV.csv')
df2 = pd.read_csv('anno_ballerTV2.csv')
df3 = pd.read_csv('anno_basketball.csv')
df4 = pd.read_csv('anno_fiba.csv')
df5 = pd.read_csv('anno_ncaa.csv')
df = pd.concat([df1, df2, df3, df4, df5], axis=0)
df.shape

(136279, 3)

In [16]:
from sklearn.model_selection import train_test_split

diff_df, eval_df = train_test_split(df, test_size=0.2)
train_df, test_df = train_test_split(diff_df, test_size=0.2)
# we reset the indices to start from zero
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
eval_df.reset_index(drop=True, inplace=True)

In [17]:
class IAMDataset(Dataset):
    def __init__(self, root_dir, df, processor, max_target_length=128):
        self.root_dir = root_dir
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
      
        file_name = self.df['file_name'][idx]
        text = str(self.df['text'][idx])
        #print(type(text))
   
        image = Image.open(file_name).convert("RGB")
        image = image.resize((64, 64))
        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        # add labels (input_ids) by encoding the text
        labels = self.processor.tokenizer(text, 
                                          padding="max_length", 
                                          max_length=self.max_target_length).input_ids
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]

        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
        return encoding

In [18]:
from transformers import TrOCRProcessor

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
train_dataset = IAMDataset(root_dir='C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\',
                           df=train_df,
                           processor=processor)
eval_dataset = IAMDataset(root_dir='C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\',
                           df=test_df,
                           processor=processor)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [19]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import requests

#processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
#model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")

In [20]:
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(eval_dataset))

Number of training examples: 87218
Number of validation examples: 21805


In [21]:
from transformers import VisionEncoderDecoderModel

model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-stage1")

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-stage1 and are newly initialized: ['encoder.pooler.dense.weight', 'encoder.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### В этом блоке мы смотрим в ручном режиме как решает нашу задачу модель из коробки

In [29]:
url = "crops\\75\\3272example.jpg"
image = Image.open(url).convert("RGB")

pixel_values = processor(image, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values)

generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(f'Номер на футболке: {generated_text}')

c:\Users\Mytre\OneDrive\Документы\Data\Work\workenv\lib\site-packages\transformers\generation\utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Номер на футболке: =


In [30]:
transform = transforms.Compose(
        [transforms.RandomResizedCrop(224), 
         transforms.ToTensor()])

In [22]:
# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size

# set beam search parameters
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 4
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 10

In [23]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    num_train_epochs=3,
    evaluation_strategy="steps",
    per_device_train_batch_size=3,
    per_device_eval_batch_size=3,
    fp16=True, 
    output_dir="./",
    logging_steps=20,
    save_steps=45000,
    eval_steps=20000,
)

In [24]:
from datasets import load_metric

cer_metric = load_metric("cer")

C:\Users\Mytre\AppData\Local\Temp\ipykernel_18976\152175726.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  cer_metric = load_metric("cer")


In [25]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(labels_ids, skip_special_tokens=True)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

In [26]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=0.001, weight_decay=0.01, no_deprecation_warning=True)


In [27]:
from transformers import default_data_collator

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=processor.feature_extractor,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=default_data_collator,
)
trainer.train()

c:\Users\Mytre\OneDrive\Документы\Data\Work\workenv\lib\site-packages\transformers\models\trocr\processing_trocr.py:134: FutureWarning: `feature_extractor` is deprecated and will be removed in v5. Use `image_processor` instead.
  warnings.warn(
c:\Users\Mytre\OneDrive\Документы\Data\Work\workenv\lib\site-packages\transformers\optimization.py:346: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/87219 [00:00<?, ?it/s]

{'loss': 4.2437, 'learning_rate': 4.998968114745641e-05, 'epoch': 0.0}
{'loss': 1.7174, 'learning_rate': 4.99782157557413e-05, 'epoch': 0.0}
{'loss': 1.6937, 'learning_rate': 4.9966750364026186e-05, 'epoch': 0.0}
{'loss': 1.8366, 'learning_rate': 4.995528497231108e-05, 'epoch': 0.0}
{'loss': 1.8606, 'learning_rate': 4.9943819580595976e-05, 'epoch': 0.0}
{'loss': 1.6611, 'learning_rate': 4.9932354188880864e-05, 'epoch': 0.0}
{'loss': 1.7437, 'learning_rate': 4.992088879716576e-05, 'epoch': 0.0}
{'loss': 1.6591, 'learning_rate': 4.990942340545065e-05, 'epoch': 0.01}
{'loss': 1.6747, 'learning_rate': 4.989795801373554e-05, 'epoch': 0.01}
{'loss': 1.6959, 'learning_rate': 4.988649262202043e-05, 'epoch': 0.01}
{'loss': 2.638, 'learning_rate': 4.9875027230305324e-05, 'epoch': 0.01}
{'loss': 1.6407, 'learning_rate': 4.986356183859022e-05, 'epoch': 0.01}
{'loss': 1.6529, 'learning_rate': 4.985209644687511e-05, 'epoch': 0.01}
{'loss': 1.7518, 'learning_rate': 4.984063105516e-05, 'epoch': 0.01}


c:\Users\Mytre\OneDrive\Документы\Data\Work\workenv\lib\site-packages\transformers\generation\utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


  0%|          | 0/7269 [00:00<?, ?it/s]

{'eval_loss': 10.180917739868164, 'eval_cer': 0.17723317350381815, 'eval_runtime': 4042.0315, 'eval_samples_per_second': 5.395, 'eval_steps_per_second': 1.798, 'epoch': 0.69}
{'loss': 0.3452, 'learning_rate': 3.853002212820601e-05, 'epoch': 0.69}
{'loss': 0.4605, 'learning_rate': 3.85185567364909e-05, 'epoch': 0.69}
{'loss': 0.5714, 'learning_rate': 3.8507091344775794e-05, 'epoch': 0.69}
{'loss': 0.3751, 'learning_rate': 3.849562595306069e-05, 'epoch': 0.69}
{'loss': 0.3566, 'learning_rate': 3.848416056134558e-05, 'epoch': 0.69}
{'loss': 0.4973, 'learning_rate': 3.847269516963047e-05, 'epoch': 0.69}
{'loss': 0.5468, 'learning_rate': 3.846122977791537e-05, 'epoch': 0.69}
{'loss': 0.3772, 'learning_rate': 3.8449764386200255e-05, 'epoch': 0.69}
{'loss': 0.4072, 'learning_rate': 3.843829899448515e-05, 'epoch': 0.69}
{'loss': 0.5886, 'learning_rate': 3.842683360277004e-05, 'epoch': 0.69}
{'loss': 0.5711, 'learning_rate': 3.841536821105493e-05, 'epoch': 0.7}
{'loss': 0.4812, 'learning_rate':

  0%|          | 0/7269 [00:00<?, ?it/s]

{'eval_loss': 11.934205055236816, 'eval_cer': 0.16313265849760256, 'eval_runtime': 3944.1809, 'eval_samples_per_second': 5.528, 'eval_steps_per_second': 1.843, 'epoch': 1.38}
{'loss': 0.3096, 'learning_rate': 2.7068643300198354e-05, 'epoch': 1.38}
{'loss': 0.3145, 'learning_rate': 2.7057177908483246e-05, 'epoch': 1.38}
{'loss': 0.4561, 'learning_rate': 2.7045712516768134e-05, 'epoch': 1.38}
{'loss': 0.3195, 'learning_rate': 2.7034247125053032e-05, 'epoch': 1.38}
{'loss': 0.2355, 'learning_rate': 2.702278173333792e-05, 'epoch': 1.38}
{'loss': 0.3468, 'learning_rate': 2.701131634162281e-05, 'epoch': 1.38}
{'loss': 0.3342, 'learning_rate': 2.6999850949907706e-05, 'epoch': 1.38}
{'loss': 0.3762, 'learning_rate': 2.6988385558192598e-05, 'epoch': 1.38}
{'loss': 0.4485, 'learning_rate': 2.6976920166477486e-05, 'epoch': 1.38}
{'loss': 0.2695, 'learning_rate': 2.6965454774762384e-05, 'epoch': 1.38}
{'loss': 0.2203, 'learning_rate': 2.6953989383047272e-05, 'epoch': 1.38}
{'loss': 0.2859, 'learni

In [91]:
url = "crops\\75\\3272example.jpg"
image = Image.open(url).convert("RGB")

pixel_values = processor(image, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values.cuda())

generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(f'Номер на футболке: {generated_text}')

c:\Users\Mytre\OneDrive\Документы\Data\Work\workenv\lib\site-packages\transformers\generation\utils.py:1288: UserWarning: Using `max_length`'s default (4) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Номер на футболке: 55
